In [ ]:
import cgmquantify as cgm
import pandas as pd
import datetime as datetime
from datetime import date
from scipy.stats import stats
import numpy as np
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
from scipy.signal import find_peaks
from scipy.stats import stats

from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LinearRegression
from numpy import mean
from numpy import absolute
from numpy import sqrt
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
!pip install cgmquantify

In [ ]:
!pip install tslearn

In [ ]:
!pip install imbalanced-learn

In [ ]:
df_merged_all = pd.read_csv('df_overnight_and_PRO.csv', sep='\t')


In [ ]:
df_merged_all = df_merged_all.dropna(subset=['axis1', 'axis2', 'axis3'],how='all')

df_merged_all['axis1'] = df_merged_all['axis1'].fillna(0)
df_merged_all['axis2'] = df_merged_all['axis2'].fillna(0)
df_merged_all['axis3'] = df_merged_all['axis3'].fillna(0)

df_merged_all['time'] = pd.to_datetime(df_merged_all['time'] ,errors = 'coerce')

df_merged_all = df_merged_all.reset_index(drop=True)
df_merged_all = df_merged_all.drop(["Unnamed: 0", "joinID"], axis=1)

fasting_states_to_keep = ['fasting', 'non-fasting']
df_merged_all = df_merged_all[df_merged_all.label.isin(fasting_states_to_keep)]

#phases_to_keep = ['lTRE', 'eTRE']
#df_merged_all = df_merged_all[df_merged_all.phase.isin(phases_to_keep)]

final_df = df_merged_all.sort_values(by=['id', 'time', "phase"], ascending=True)

In [ ]:
final_df = final_df.reset_index(drop=True)

## Label Encoding

In [ ]:
map_dict = {"non-fasting": 1, "fasting":0}
df_train["label"] = df_train["label"].map(map_dict)
df_test["label"] = df_test["label"].map(map_dict)

In [ ]:
map_dict = {"screening": 1, "lTRE":0, "eTRE":2}
df_train["phase"] = df_train["phase"].map(map_dict)
df_test["phase"] = df_test["phase"].map(map_dict)

In [ ]:
df_train

## Saving and Loading the featured Data

In [ ]:
train_labels = df_train["label"]
test_labels = df_test["label"]

In [ ]:
y_train = np.array(train_labels)
y_test = np.array(test_labels)

In [ ]:
y_train_float = y_train
y_test_float = y_test

In [ ]:
mylist = list(set(y_train_float))
mylist

## Building the time series

### Test-dataset

In [ ]:
X_test = df_test

### acc and gl

In [ ]:
from datetime import datetime


X_test.time = pd.to_datetime(X_test.time)

x_list_test_tsc_1st = []
test_labels_tsc = []
x_list_test_tsc_2nd = []
counter=0


last_time_before = X_test['time'].values[0]

for i in range(0, X_test.shape[0], 1):
    time = X_test['time'].values[i]
    gl =  X_test['gl'].values[i]
    axis1 = X_test['axis1'].values[i]
    axis2 = X_test['axis2'].values[i]
    axis3 = X_test['axis3'].values[i]
    steps = X_test['steps'].values[i]
    lux = X_test['lux'].values[i]
    inclineStanding = X_test['inclineStanding'].values[i]
    inclineSitting = X_test['inclineSitting'].values[i]
    inclineLying = X_test['inclineLying'].values[i]
    vm = X_test['vm'].values[i]

    timedelta_first =  time - last_time_before

    one_hour = 3600000000000
    
    if timedelta_first < one_hour and counter < 5:
        x_list_test_tsc_3rd = []
        x_list_test_tsc_3rd.append(gl)
        x_list_test_tsc_3rd.append(axis1)
        x_list_test_tsc_3rd.append(axis2)
        x_list_test_tsc_3rd.append(axis3)
        x_list_test_tsc_3rd.append(steps)
        x_list_test_tsc_3rd.append(lux)
        x_list_test_tsc_3rd.append(inclineStanding)
        x_list_test_tsc_3rd.append(inclineSitting)
        x_list_test_tsc_3rd.append(inclineLying)
        x_list_test_tsc_3rd.append(vm)

        x_list_test_tsc_2nd.append(x_list_test_tsc_3rd)
    
    else:
        
        if counter == 5:
            x_list_test_tsc_1st.append(x_list_test_tsc_2nd)
            test_labels_tsc.append(y_test_float[i])
            
        counter = 0
        x_list_test_tsc_2nd = [] 
        x_list_test_tsc_3rd = []
        x_list_test_tsc_3rd.append(gl)
        x_list_test_tsc_3rd.append(axis1)
        x_list_test_tsc_3rd.append(axis2)
        x_list_test_tsc_3rd.append(axis3)
        x_list_test_tsc_3rd.append(steps)
        x_list_test_tsc_3rd.append(lux)
        x_list_test_tsc_3rd.append(inclineStanding)
        x_list_test_tsc_3rd.append(inclineSitting)
        x_list_test_tsc_3rd.append(inclineLying)
        x_list_test_tsc_3rd.append(vm)
        x_list_test_tsc_2nd.append(x_list_test_tsc_3rd)
        
    counter += 1
    
    last_time_before = time


### gl

In [ ]:
from datetime import datetime


X_test.time = pd.to_datetime(X_test.time)

x_list_test_tsc_1st = []
test_labels_tsc = []
x_list_test_tsc_2nd = []
counter=0


last_time_before = X_test['time'].values[0]

for i in range(0, X_test.shape[0], 1):
    time = X_test['time'].values[i]
    gl =  X_test['gl'].values[i]

    
    timedelta_first =  time - last_time_before

    one_hour = 3600000000000
    
    if timedelta_first < one_hour and counter < 5:
        x_list_test_tsc_3rd = []
        x_list_test_tsc_3rd.append(gl)

        x_list_test_tsc_2nd.append(x_list_test_tsc_3rd)
    
    else:
        
        if counter == 5:
            x_list_test_tsc_1st.append(x_list_test_tsc_2nd)
            test_labels_tsc.append(y_test_float[i])
            
        counter = 0
        x_list_test_tsc_2nd = [] 
        x_list_test_tsc_3rd = []
        x_list_test_tsc_3rd.append(gl)
        x_list_test_tsc_2nd.append(x_list_test_tsc_3rd)
        
    counter += 1
    
    last_time_before = time

### acc

In [ ]:
from datetime import datetime


X_test.time = pd.to_datetime(X_test.time)

x_list_test_tsc_1st = []
test_labels_tsc = []
x_list_test_tsc_2nd = []
counter=0


last_time_before = X_test['time'].values[0]

for i in range(0, X_test.shape[0], 1):
    time = X_test['time'].values[i]
    axis1 = X_test['axis1'].values[i]
    axis2 = X_test['axis2'].values[i]
    axis3 = X_test['axis3'].values[i]
    steps = X_test['steps'].values[i]
    lux = X_test['lux'].values[i]
    inclineStanding = X_test['inclineStanding'].values[i]
    inclineSitting = X_test['inclineSitting'].values[i]
    inclineLying = X_test['inclineLying'].values[i]
    vm = X_test['vm'].values[i]
    
    timedelta_first =  time - last_time_before

    one_hour = 3600000000000
    
    if timedelta_first < one_hour and counter < 5:
        x_list_test_tsc_3rd = []

        x_list_test_tsc_3rd.append(axis1)
        x_list_test_tsc_3rd.append(axis2)
        x_list_test_tsc_3rd.append(axis3)
        x_list_test_tsc_3rd.append(steps)
        x_list_test_tsc_3rd.append(lux)
        x_list_test_tsc_3rd.append(inclineStanding)
        x_list_test_tsc_3rd.append(inclineSitting)
        x_list_test_tsc_3rd.append(inclineLying)
        x_list_test_tsc_3rd.append(vm)
        x_list_test_tsc_2nd.append(x_list_test_tsc_3rd)
    
    else:
        
        if counter == 5:
            x_list_test_tsc_1st.append(x_list_test_tsc_2nd)
            test_labels_tsc.append(y_test_float[i])
            
        counter = 0
        x_list_test_tsc_2nd = [] 
        x_list_test_tsc_3rd = []

        x_list_test_tsc_3rd.append(axis1)
        x_list_test_tsc_3rd.append(axis2)
        x_list_test_tsc_3rd.append(axis3)
        x_list_test_tsc_3rd.append(steps)
        x_list_test_tsc_3rd.append(lux)
        x_list_test_tsc_3rd.append(inclineStanding)
        x_list_test_tsc_3rd.append(inclineSitting)
        x_list_test_tsc_3rd.append(inclineLying)
        x_list_test_tsc_3rd.append(vm)
        x_list_test_tsc_2nd.append(x_list_test_tsc_3rd)
        
    counter += 1
    
    last_time_before = time

### Train-Dataset

In [ ]:
X_train = df_train

In [ ]:
from datetime import datetime


X_train.time = pd.to_datetime(X_train.time)


x_list_train_tsc_1st = []
train_labels_tsc = []
x_list_train_tsc_2nd = []
counter = 0


last_time_before = X_train['time'].values[0]



for i in range(0, X_train.shape[0], 1):
    time = X_train['time'].values[i]
    gl =  X_train['gl'].values[i]
    axis1 = X_train['axis1'].values[i]
    axis2 = X_train['axis2'].values[i]
    axis3 = X_train['axis3'].values[i]
    steps = X_train['steps'].values[i]
    lux = X_train['lux'].values[i]
    inclineStanding = X_train['inclineStanding'].values[i]
    inclineSitting = X_train['inclineSitting'].values[i]
    inclineLying = X_train['inclineLying'].values[i]
    vm = X_train['vm'].values[i]
    
    timedelta_first =  time - last_time_before

    one_hour = 3600000000000
    
    if timedelta_first < one_hour and counter < 5:
        
        x_list_train_tsc_3rd = []
        x_list_train_tsc_3rd.append(gl)
        x_list_train_tsc_3rd.append(axis1)
        x_list_train_tsc_3rd.append(axis2)
        x_list_train_tsc_3rd.append(axis3)
        x_list_train_tsc_3rd.append(steps)
        x_list_train_tsc_3rd.append(lux)
        x_list_train_tsc_3rd.append(inclineStanding)
        x_list_train_tsc_3rd.append(inclineSitting)
        x_list_train_tsc_3rd.append(inclineLying)
        x_list_train_tsc_3rd.append(vm)
        x_list_train_tsc_2nd.append(x_list_train_tsc_3rd)
    
    else:
        if counter == 5:
            x_list_train_tsc_1st.append(x_list_train_tsc_2nd)
            train_labels_tsc.append(y_train_float[i])
            
        counter = 0
        x_list_train_tsc_2nd = [] 
        x_list_train_tsc_3rd = []
        x_list_train_tsc_3rd.append(gl)
        x_list_train_tsc_3rd.append(axis1)
        x_list_train_tsc_3rd.append(axis2)
        x_list_train_tsc_3rd.append(axis3)
        x_list_train_tsc_3rd.append(steps)
        x_list_train_tsc_3rd.append(lux)
        x_list_train_tsc_3rd.append(inclineStanding)
        x_list_train_tsc_3rd.append(inclineSitting)
        x_list_train_tsc_3rd.append(inclineLying)
        x_list_train_tsc_3rd.append(vm)
        x_list_train_tsc_2nd.append(x_list_train_tsc_3rd)
    counter += 1
    last_time_before = time

### gl

In [ ]:
from datetime import datetime


X_train.time = pd.to_datetime(X_train.time)


x_list_train_tsc_1st = []
train_labels_tsc = []
x_list_train_tsc_2nd = []
counter = 0


last_time_before = X_train['time'].values[0]



for i in range(0, X_train.shape[0], 1):
    time = X_train['time'].values[i]
    gl =  X_train['gl'].values[i]
    timedelta_first =  time - last_time_before

    one_hour = 3600000000000
    
    if timedelta_first < one_hour and counter < 5:
        
        x_list_train_tsc_3rd = []
        x_list_train_tsc_3rd.append(gl)

        x_list_train_tsc_2nd.append(x_list_train_tsc_3rd)
    
    else:
        if counter == 5:
            x_list_train_tsc_1st.append(x_list_train_tsc_2nd)
            train_labels_tsc.append(y_train_float[i])
            
        counter = 0
        x_list_train_tsc_2nd = [] 
        x_list_train_tsc_3rd = []
        x_list_train_tsc_3rd.append(gl)

        x_list_train_tsc_2nd.append(x_list_train_tsc_3rd)
    counter += 1
    last_time_before = time

### acc

In [ ]:
from datetime import datetime


X_train.time = pd.to_datetime(X_train.time)


x_list_train_tsc_1st = []
train_labels_tsc = []
x_list_train_tsc_2nd = []
counter = 0


last_time_before = X_train['time'].values[0]



for i in range(0, X_train.shape[0], 1):
    time = X_train['time'].values[i]
    axis1 = X_train['axis1'].values[i]
    axis2 = X_train['axis2'].values[i]
    axis3 = X_train['axis3'].values[i]
    steps = X_train['steps'].values[i]
    lux = X_train['lux'].values[i]
    inclineStanding = X_train['inclineStanding'].values[i]
    inclineSitting = X_train['inclineSitting'].values[i]
    inclineLying = X_train['inclineLying'].values[i]
    vm = X_train['vm'].values[i]
    
    timedelta_first =  time - last_time_before

    one_hour = 3600000000000
    
    if timedelta_first < one_hour and counter < 5:
        
        x_list_train_tsc_3rd = []
        x_list_train_tsc_3rd.append(axis1)
        x_list_train_tsc_3rd.append(axis2)
        x_list_train_tsc_3rd.append(axis3)
        x_list_train_tsc_3rd.append(steps)
        x_list_train_tsc_3rd.append(lux)
        x_list_train_tsc_3rd.append(inclineStanding)
        x_list_train_tsc_3rd.append(inclineSitting)
        x_list_train_tsc_3rd.append(inclineLying)
        x_list_train_tsc_3rd.append(vm)
        x_list_train_tsc_2nd.append(x_list_train_tsc_3rd)
    
    else:
        if counter == 5:
            x_list_train_tsc_1st.append(x_list_train_tsc_2nd)
            train_labels_tsc.append(y_train_float[i])
            
        counter = 0
        x_list_train_tsc_2nd = [] 
        x_list_train_tsc_3rd = []
        x_list_train_tsc_3rd.append(axis1)
        x_list_train_tsc_3rd.append(axis2)
        x_list_train_tsc_3rd.append(axis3)
        x_list_train_tsc_3rd.append(steps)
        x_list_train_tsc_3rd.append(lux)
        x_list_train_tsc_3rd.append(inclineStanding)
        x_list_train_tsc_3rd.append(inclineSitting)
        x_list_train_tsc_3rd.append(inclineLying)
        x_list_train_tsc_3rd.append(vm)
        
        x_list_train_tsc_2nd.append(x_list_train_tsc_3rd)
    counter += 1
    last_time_before = time

# TS Learn Classification

### To Time Series Dataset

In [ ]:
from tslearn.utils import to_time_series_dataset
from tslearn.neighbors import KNeighborsTimeSeriesClassifier
from tslearn import generators


In [ ]:
X_train =to_time_series_dataset(x_list_train_tsc_1st)
X_test =to_time_series_dataset(x_list_test_tsc_1st)

In [ ]:
y_train =  np.array(train_labels_tsc)
y_test =  np.array(test_labels_tsc)

### Checking for Balance

In [ ]:
from collections import Counter
print(Counter(y_test).keys()) # equals to list(set(words))
print(Counter(y_test).values()) # counts the elements' frequency
print(Counter(y_train).keys()) # equals to list(set(words))
print(Counter(y_train).values()) # counts the elements' frequency

In [ ]:
data2d_test = X_test.reshape(X_test.shape[0], -1)
data2d_train = X_train.reshape(X_train.shape[0], -1)

In [ ]:
print(X_test.shape)
print(X_train.shape)
print(data2d_test.shape)
print(data2d_train.shape)

### Undersampling with Random Selection

In [ ]:
from imblearn.under_sampling import RandomUnderSampler
import collections
rus = RandomUnderSampler(random_state=0, replacement=True)
data2d_test_resampled , y_test_resampled = rus.fit_resample(data2d_test, y_test)
data2d_train_resampled , y_train_resampled = rus.fit_resample(data2d_train, y_train)

In [ ]:
print(data2d_test_resampled.shape)
print(data2d_train_resampled.shape)

In [ ]:
from collections import Counter
print(Counter(y_test_resampled).keys()) # equals to list(set(words))
print(Counter(y_test_resampled).values()) # counts the elements' frequency
print(Counter(y_train_resampled).keys()) # equals to list(set(words))
print(Counter(y_train_resampled).values()) # counts the elements' frequency

In [ ]:
data2d_test_resampled.shape[0]

In [ ]:
scaler = MinMaxScaler()
scaler.fit(data2d_train_resampled)
data2d_train_resampled = scaler.transform(data2d_train_resampled)
data2d_test_resampled = scaler.transform(data2d_test_resampled)

In [ ]:
data3d_X_test = data2d_test_resampled.reshape(-1, X_test.shape[1], X_test.shape[2])
data3d_X_train = data2d_train_resampled.reshape(-1, X_train.shape[1], X_train.shape[2])

In [ ]:
print(data3d_X_test.shape)
print(data3d_X_train.shape)

In [ ]:
def GridSearch_table_plot(grid_clf, param_name,
                          num_results=15,
                          negative=True,
                          graph=True,
                          display_all_params=True):

    '''Display grid search results

    Arguments
    ---------

    grid_clf           the estimator resulting from a grid search
                       for example: grid_clf = GridSearchCV( ...

    param_name         a string with the name of the parameter being tested

    num_results        an integer indicating the number of results to display
                       Default: 15

    negative           boolean: should the sign of the score be reversed?
                       scoring = 'neg_log_loss', for instance
                       Default: True

    graph              boolean: should a graph be produced?
                       non-numeric parameters (True/False, None) don't graph well
                       Default: True

    display_all_params boolean: should we print out all of the parameters, not just the ones searched for?
                       Default: True

    Usage
    -----

    GridSearch_table_plot(grid_clf, "min_samples_leaf")

                          '''
    from matplotlib      import pyplot as plt
    from IPython.display import display
    import pandas as pd

    clf = grid_clf.best_estimator_
    clf_params = grid_clf.best_params_
    if negative:
        clf_score = -grid_clf.best_score_
    else:
        clf_score = grid_clf.best_score_
    clf_stdev = grid_clf.cv_results_['std_test_score'][grid_clf.best_index_]
    cv_results = grid_clf.cv_results_

    print("best parameters: {}".format(clf_params))
    print("best score:      {:0.5f} (+/-{:0.5f})".format(clf_score, clf_stdev))
    if display_all_params:
        import pprint
        pprint.pprint(clf.get_params())

    # pick out the best results
    # =========================
    scores_df = pd.DataFrame(cv_results).sort_values(by='rank_test_score')

    best_row = scores_df.iloc[0, :]
    if negative:
        best_mean = -best_row['mean_test_score']
    else:
        best_mean = best_row['mean_test_score']
    best_stdev = best_row['std_test_score']
    best_param = best_row['param_' + param_name]

    # display the top 'num_results' results
    # =====================================
    display(pd.DataFrame(cv_results) \
            .sort_values(by='rank_test_score').head(num_results))

    # plot the results
    # ================
    scores_df = scores_df.sort_values(by='param_' + param_name)

    if negative:
        means = -scores_df['mean_test_score']
    else:
        means = scores_df['mean_test_score']
    stds = scores_df['std_test_score']
    params = scores_df['param_' + param_name]

    # plot
    if graph:
        plt.figure(figsize=(8, 8))
        plt.errorbar(params, means, yerr=stds)

        plt.axhline(y=best_mean + best_stdev, color='red')
        plt.axhline(y=best_mean - best_stdev, color='red')
        plt.plot(best_param, best_mean, 'or')

        plt.title(param_name + " vs Score\nBest Score {:0.5f}".format(clf_score))
        plt.xlabel(param_name)
        plt.ylabel('Score')
        plt.show()

In [ ]:
from numpy.lib.stride_tricks import sliding_window_view


def smoothing(y_pred):
    
    outcomes = np.array([])
    last_four_values = y_pred[-4:]
    
    y_pred_windowed = sliding_window_view((y_pred), window_shape = 5)

    i = 0
    for window in y_pred_windowed:
        pred_one = np.count_nonzero(window == 1)
        pred_zero = 5 - pred_one
        if pred_one > pred_zero:
            outcome = int(1)
            outcomes = np.append(outcomes, outcome)
        else:
            outcome = int(0)
            outcomes = np.append(outcomes, outcome)
    
    outcomes = np.append(outcomes, last_four_values)
            
    
    return outcomes

## KNeighborsTimeSeriesClassifier

In [ ]:
from sklearn.model_selection import RandomizedSearchCV


In [ ]:
from tslearn.neighbors import KNeighborsTimeSeriesClassifier
knn = KNeighborsTimeSeriesClassifier(n_neighbors=93)
knn.fit(data3d_X_train, y_train_resampled)

In [ ]:
y_pred = knn.predict(data3d_X_test)

In [ ]:
print("Accuracy:", accuracy_score(y_test_resampled,smoothing(y_pred)))
print("\n -------------Classification Report-------------\n")
print(classification_report(y_test_resampled, y_pred))

In [ ]:
# set search parameters
n_neighbors = [int(x) for x in np.linspace(start = 1, stop = 100, num = 50)]   
weights = ['uniform','distance']
metric = ['euclidean','manhattan','chebyshev','minkowski'] 
random_grid = {
    'n_neighbors': n_neighbors,
    'weights': weights,
    'metric': metric,
}

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
knn_random = RandomizedSearchCV(estimator = knn, random_state = 42,n_jobs = -1,param_distributions = random_grid,n_iter = 100, cv=3,verbose = 2)
knn_random.fit(data3d_X_train, y_train_resampled)
knn_random.best_params_

In [ ]:
knn_random.best_params_

best_estimator = knn_random.best_estimator_

best_estimator.fit(data3d_X_train, y_train_resampled)

pred = best_estimator.predict(data3d_X_test)

accuracy_score(pred, y_test_resampled)

In [ ]:
print("\n -------------Classification Report-------------\n")
print(classification_report(y_test_resampled, pred))

In [ ]:
GridSearch_table_plot(knn_random, "n_neighbors", negative=False)

In [ ]:
y_pred = knn.predict(data3d_X_test)
accuracy_score(y_test_resampled, y_pred)

In [ ]:
from sklearn import metrics

confusion_matrix = metrics.confusion_matrix(y_test_resampled, y_pred)

cm_display = metrics.ConfusionMatrixDisplay(confusion_matrix = confusion_matrix, display_labels = [False, True])

cm_display.plot()


In [ ]:
import pickle
with open('TSC_data/models_raw_TSC/gl/KNN_TSC_scaled.pkl','wb') as f:
    pickle.dump(knn,f)

## TimeSeriesSVC

In [ ]:
import pickle
start = "models_raw_TSC\\"
end = ".pkl"
folder = "gl\\"
ml_type = "KNN_TSC"

path = start + folder + ml_type + end

with open(path, "rb") as f:
    model = pickle.load(f)

In [ ]:
from tslearn.svm import TimeSeriesSVC
clf = TimeSeriesSVC(C=1.0, kernel="gak")
clf.fit(data3d_X_train,y_train_resampled)

In [ ]:
parameters = {'kernel': ['rbf', 'sigmoid'],
             'C': [1.0, 100.0, 1000.0],
             'gamma': [0.001, 0.01,0.1]}#


In [ ]:
random_estimator = RandomizedSearchCV(estimator = clf,
                                   param_distributions = parameters,
                                   cv = 5,
                                   n_iter = 10,
                                   scoring = 'accuracy',
                                   n_jobs = -1,
                                   verbose = 1, 
                                   random_state = 1,
                                  )

random_estimator.fit(data3d_X_train, y_train_resampled)

In [ ]:
random_estimator.best_params_

In [ ]:
random_estimator.best_params_

best_estimator = random_estimator.best_estimator_

best_estimator.fit(data3d_X_train, y_train_resampled)

pred = best_estimator.predict(data3d_X_test)

accuracy_score(pred, smoothing(y_test_resampled))

In [ ]:
pred = model.predict(data3d_X_test)

accuracy_score(y_test_resampled, smoothing(pred))

In [ ]:
print("\n -------------Classification Report-------------\n")
print(classification_report(y_test_resampled, pred))

In [ ]:
y_pred = clf.predict(data3d_X_test)
accuracy_score(y_test_resampled, smoothing(y_pred))

In [ ]:
parameters = {'kernel': ['rbf', 'sigmoid', 'poly'],
             'C': [0.1, 1.0, 100.0, 1000.0],
             'gamma': [0.00001, 0.0001, 0.001, 0.01, 0.1, 1.0],}#


In [ ]:
random_estimator = RandomizedSearchCV(estimator = clf,
                                   param_distributions = parameters,
                                   cv = 5,
                                   n_iter = 100,
                                   scoring = 'accuracy',
                                   n_jobs = -1,
                                   verbose = 1, 
                                   random_state = 1,
                                  )

random_estimator.fit(data3d_X_train, y_train_resampled)

In [ ]:
random_estimator.best_params_

best_estimator = random_estimator.best_estimator_

best_estimator.fit(X_train_data_lr, y_train_resampled)

pred = best_estimator.predict(X_test_data_lr)

accuracy_score(pred, y_test_resampled)

In [ ]:
GridSearch_table_plot(random_estimator, "gamma", negative=False)

In [ ]:
from sklearn import metrics
print(metrics.classification_report(y_test_resampled, pred))

In [ ]:
from sklearn import metrics

confusion_matrix = metrics.confusion_matrix(y_test_resampled, y_pred)

cm_display = metrics.ConfusionMatrixDisplay(confusion_matrix = confusion_matrix, display_labels = [False, True])

cm_display.plot()


In [ ]:
import pickle
with open('TSC_data/models_raw_TSC/gl/CLF_TSC.pkl','wb') as f:
    pickle.dump(clf,f)

In [ ]:
print("d")

## Clustering TimeSeriesKMeans

In [ ]:
from tslearn.clustering import TimeSeriesKMeans
km = TimeSeriesKMeans(n_clusters=2, metric="dtw")
labels = km.fit_predict(data3d_X_train)


km_bis = TimeSeriesKMeans(n_clusters=2, metric="softdtw")
labels_bis = km_bis.fit_predict(data3d_X_train)

In [ ]:
pred_df = pd.DataFrame({"actual":y_train_resampled, "prediction":labels})

In [ ]:
correct = 0
for i in range(len(y_train_resampled)):
    if labels_bis[i] == y_train_resampled[i]:
        correct += 1

print(correct/len(y_train_resampled))

## Model Selection with hypterparameter tuning and KNeighborsTimeSeriesClassifier

In [ ]:
from sklearn.model_selection import KFold, GridSearchCV
from tslearn.neighbors import KNeighborsTimeSeriesClassifier

knn = KNeighborsTimeSeriesClassifier(metric="dtw")
p_grid = {"n_neighbors": [1, 5]}

cv = KFold(n_splits=2, shuffle=True, random_state=0)
clf = GridSearchCV(estimator=knn, param_grid=p_grid, cv=cv)
clf.fit(data3d_X_train, y_train_resampled)